# Plot harmonic measure on a GW-tree as apparent after some finite number of generations

In [ ]:
import networkx as nx
import markovmixing as mkm

import numpy

execfile('graph_util.py')

## Choose one of these offspring distributions

In [ ]:
offspring = lambda: numpy.random.poisson(1.5)
generation_measure = 8
generation_hit = 13

In [ ]:
offspring = lambda: numpy.random.choice([1,2])
generation_measure = 10
generation_hit = 20

In [ ]:
# as in http://arxiv.org/abs/1504.01999
offspring = lambda: numpy.random.choice([1,3])
generation_measure = 7
generation_hit = 14

## Generate a GW-tree with the chosen offspring

In [ ]:
numpy.random.seed(0)

G = grow_gw_tree(offspring, n_gen = generation_hit, directed = True)

while nx.number_of_nodes(G) <= 100: 
    G = grow_gw_tree(offspring, n_gen = generation_hit, directed = True)
    
print nx.number_of_nodes(G)

In [ ]:
# Show the tree
show_tree(G)

## Setup for the simulation

In [ ]:
# The tree up to the generation where we want to determine the harmonic measure
G_measure = G.copy()

for node in G_measure.nodes():
	if G_measure.node[node]['generation']>generation_measure:
			G_measure.remove_node(node)
            
# the nodes of generation measure and hit
generation = nx.get_node_attributes(G, 'generation')

generation_measure_nodes = [key for key in generation if generation[key] == generation_measure]
generation_hit_nodes = [key for key in generation if generation[key] == generation_hit]

# dictionary mapping nodes of generation hit to their ancestors in generation measure
ancestor_dict = {}

for nh in generation_hit_nodes:
    a = nh
    
    while generation[a] != generation_measure:
        a = G.predecessors(a)[0] 
        
    ancestor_dict[nh] = a
        
print generation_measure_nodes
print len(generation_hit_nodes)
print nx.number_of_nodes(G_measure)

In [ ]:
# Show the tree up to generation measure
show_tree_debug(G_measure)

## Monte-Carlo simulation of SRW on the tree

In [ ]:
numpy.random.seed(0)

# monte-carlo simulation of the harmonic measure 
mc = mkm.nx_graph_srw(G.to_undirected())
srw_hits = {}

for n in generation_measure_nodes:
    srw_hits[n] = 0

for i in xrange(100000):
    if i % 1000 == 0:
        print i
    
    # start a SRW at the root
    path = [0]
    hit = -1

    while hit == -1:
        # let the SRW walk 99 steps
        path = mc.sample_path(path[-1],100)

        # did we hit generation hit?
        for x in path:
            if generation[x] == generation_hit:
                hit = x
                break
    
    # log the hit
    srw_hits[ancestor_dict[hit]] = srw_hits[ancestor_dict[hit]]+1

print srw_hits
print sum(srw_hits.values())

### Plot the distribution of hits

In [ ]:
import matplotlib.pyplot as plt

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

x = srw_hits.keys()
n = sum(srw_hits.values())*1.0

y = []
for key in x:
    y.append(srw_hits[key]/n)
    
fig = plt.figure()

vmax = numpy.max(y)
vmin = (numpy.min(y)*3. - vmax)/2.

colormap = ScalarMappable(norm=Normalize(vmin, vmax), cmap='Blues')

plt.bar(numpy.arange(len(y)), y, color=colormap.to_rgba(y), align='edge', width=1.0)

plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')

plt.xlim(0, len(x))
plt.show()

## Monte-Carlo simulation of NBRW on the tree

In [ ]:
numpy.random.seed(0)

A = nx.to_scipy_sparse_matrix(G.to_undirected())
P = mkm.tree_nbrw_transition_matrix(A, 0)
mc = mkm.MarkovChain(P)

nbrw_hits = {}

for n in generation_measure_nodes:
    nbrw_hits[n] = 0

for i in xrange(50000):
    if i % 1000 == 0:
        print i
    
    # start a NBRW at the root
    path = [0]
    hit = -1
    
    # let the NBRW walk to generation_hit
    path = mc.sample_path(path[-1],generation_hit+1)
    assert(generation[path[-1]] == generation_hit)

    # log the hit
    nbrw_hits[ancestor_dict[path[-1]]] = nbrw_hits[ancestor_dict[path[-1]]]+1

print nbrw_hits
print sum(nbrw_hits.values())

### Plot the distribution of hits

In [ ]:
import matplotlib.pyplot as plt

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

x = nbrw_hits.keys()
n = sum(nbrw_hits.values())*1.0

y = []
for key in x:
    y.append(nbrw_hits[key]/n)
    
fig = plt.figure()

vmax = numpy.max(y)
vmin = (numpy.min(y)*3. - vmax)/2.

colormap = ScalarMappable(norm=Normalize(vmin, vmax), cmap='Blues')

plt.bar(numpy.arange(len(y)), y, color=colormap.to_rgba(y), align='edge', width=1.0)

plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')

plt.xlim(0, len(x))
plt.show()